In [1]:
setwd('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1002. Ecig Metabolomics NELF/P1002.3. Analyses/P1002.3.3. Summary Statistics/Input')
Output = ('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1002.3. Analyses/P1002.3.3. Summary Statistics/Output')
cur_date = "030322"

library(readxl)
library(tidyverse)
library(reshape2)
library(car)


#reading in file
demographics_df = data.frame(read_excel("SubjectInfo_022822.xlsx", sheet = 2)) 

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.6     ✔ dplyr   1.0.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode


The following object is masked from ‘package:purrr’:

    some




In [2]:
head(demographics_df)

,Original_SubjectID,SubjectID,Exposure,Sex,Race,Age,BMI
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,NBL214,Ecig_214,Ecig,F,O,19,20.9
2,NBL221,Ecig_221,Ecig,M,As,21,24.0
3,NBL222,Ecig_222,Ecig,F,W,20,30.2
4,NBL223,Ecig_223,Ecig,F,B,19,19.9
5,NBL229,Ecig_229,Ecig,F,W,39,35.1
6,NBL230,Ecig_230,Ecig,F,W,40,45.6


# Chi Square Analysis/ ANOVA

Comparing demographic variables to see if there are differences across smoking statuses. 

In [3]:
get_tables = function(df, variable){
    #getting variable stratified by smoking status
    variable_grouped = df %>% group_by(Exposure) %>% select(all_of(variable))
    #getting overall variable numbers
    overall_variable = df %>% select(all_of(variable)) 
    #adding a col that designates this from the overall cohort
    overall_variable_df = cbind(Exposure = rep('O', times = length(overall_variable)), 
                                           overall_variable)
    #for some reason this col was a factor and need to make it a character for subsequent lines to work
    overall_variable_df$Exposure = as.character(overall_variable_df$Exposure)
    

    final_df = rbind(overall_variable_df, data.frame(variable_grouped))
    variable_table = table(final_df)
    return(variable_table)
}

#calling fn
sex_table = get_tables(demographics_df, 'Sex')
race_table = get_tables(demographics_df, 'Race')

#viewing results
sex_table
race_table

Adding missing grouping variables: `Exposure`

Adding missing grouping variables: `Exposure`



        Sex
Exposure  F  M  W
    Ecig 20 12  0
    NS   14 12  1
    O    34 24  1

        Race
Exposure 29 As  B  O  W
    Ecig  0  3  5  2 22
    NS    1  3  4  0 19
    O     1  6  9  2 41

In [4]:
#fisher exact test
fisher.test(sex_table)
fisher.test(race_table)


	Fisher's Exact Test for Count Data

data:  sex_table
p-value = 0.8189
alternative hypothesis: two.sided



	Fisher's Exact Test for Count Data

data:  race_table
p-value = 0.9759
alternative hypothesis: two.sided


In [5]:
#dichotimized race
#making race binary between white and non-white subjects
for (i in 1:length(demographics_df$Race)){
    if(demographics_df$Race[i] != 'W'){
        demographics_df$Race[i] = 'NW'
    }
}

#dichotimized race
di_race_table = get_tables(demographics_df, 'Race')
di_race_table
fisher.test(di_race_table)

Adding missing grouping variables: `Exposure`



        Race
Exposure NW  W
    Ecig 10 22
    NS    8 19
    O    18 41


	Fisher's Exact Test for Count Data

data:  di_race_table
p-value = 1
alternative hypothesis: two.sided


Since none of the p values are significant, the demographics between each smoking status isn't statistically different.

In [6]:
#running anova test for continuous values: age, bmi, cotinine

#putting Exposure into a factor in order to make comparisons
demographics_df$Exposure = factor(demographics_df$Exposure, labels = c(unique(demographics_df$Exposure)))

#normalizing data
mutated_df = demographics_df %>%
    mutate(Norm_Age = scale(Age), Norm_BMI = scale(BMI))
#contrasts show what compartments are being compared in anova
#these comparisons aren't what we want so I changed them in the function below
contrasts(mutated_df$Exposure)

,NS
Ecig,0
NS,1


                   Table of Contrasts
>                   NS    |  Ecig  |  Sum
>     Contrast 1 |  -1    |   1    |   0

> - Contrast 1: compares NS to Ecig

In [7]:
#contrasts(mutated_df$Exposure) = cbind(c(-1,1,0)) #meaning is specified above
anova_age = Anova(aov(Norm_Age ~ Exposure, data = mutated_df), type = 'III')
anova_age
anova_bmi = Anova(aov(Norm_BMI ~ Exposure, data = mutated_df), type = 'III')
anova_bmi

,Sum Sq,Df,F value,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),5.827716,1,7.338498,0.0088968382
Exposure,12.734638,1,16.035978,0.0001818886
Residuals,45.265362,57,NA,NA


,Sum Sq,Df,F value,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.6954052,1,0.7023191,0.4055683
Exposure,1.5512884,1,1.5667118,0.2158884
Residuals,55.4487116,56,NA,NA
